# Tuning the Top-k Query parameter

First we set up the environment

In [5]:
from dotenv import load_dotenv
import os
import psycopg2
import pandas as pd
from time import time
import torch
from Enrichers import Enricher, get_enricher
from database.database import DatabaseProcessor
from Embedders import Embedder, get_embedder
from tqdm import tqdm

load_dotenv('.env', override=True)

# Database setup
db_params = {
    'dbname': os.getenv('DB_NAME'),
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD'),
    'host': os.getenv('DB_HOST'),
    'port': os.getenv('DB_PORT')
}
db = DatabaseProcessor(db_params)
db.test_connection()

device = 'cuda' if torch.cuda.is_available(
) else 'mps' if torch.mps.is_available() else 'cpu'
print(f"Using device: {device}")

=================================CONFIG=================================
Database         User             Host                             Port            
citeline_db      bbasseri         localhost                        5432            
Database version: ('PostgreSQL 17.3 (Homebrew) on x86_64-apple-darwin23.6.0, compiled by Apple clang version 16.0.0 (clang-1600.0.26.6), 64-bit',)
Using device: mps


## Investigating precision over `k`

For our various embedding models and enrichment strategies, we want to know the smallest `top_k` value that will still retrieve the target reference for a given sentence. 

To investigate this, we'll sample 100 examples from the non-trivial training data. Each example typically has 1-2 target DOI's. For each example, we'll query the database with a large `top_k` parameter to start, so we can be sure the database returns the target references. Then we can ask at what index in the query results does a target DOI first appear. Ideally, the ranks will all be very high, indicated by having *low* indices in the query results. We also expect enriched examples to have their target doi's higher ranked (lower indices).

In [6]:
from database.database import SingleQueryResult

data = pd.read_json('data/dataset/split/train.jsonl', lines=True)
examples = data.sample(100, random_state=42)

In [ ]:
def lowest_index_matching_doi(target_doi: str, query_results: list[SingleQueryResult]) -> int:
    for i, result in enumerate(query_results):
        if target_doi == result.doi:
            return i
    return -1


def get_ranks(
    example: pd.Series,
    embedder: Embedder,
    enricher: Enricher,
    table_name: str,
    top_k:
    int,
    metric: str = 'vector_cosine_ops'
) -> list[int]:
    target_dois = example['citation_dois']

    # Prepare query vector
    enriched_sentence = enricher.enrich(example=example)
    embedding = embedder([enriched_sentence])[0]

    # Query
    query_results = db.query_vector_table(
        table_name=table_name,
        query_vector=embedding,
        metric=metric,
        use_index=True,
        top_k=top_k
    )
    ranks = [lowest_index_matching_doi(
        target_doi=doi, query_results=query_results) for doi in target_dois]
    return ranks

In [7]:
embedder = get_embedder(model_name='BAAI/bge-small-en',
                        device=device, normalize=True)
enricher = get_enricher("identity")

embeddings = embedder(enricher.enrich_batch(examples))  # Enrich and embed the entire batch

In [8]:
for embedding in tqdm(embeddings):
    start = time()
    query_result = db.query_vector_table(
        table_name='bge',
        query_vector=embedding,
        metric='vector_cosine_ops',
        use_index=True,
        top_k=1000
    )
    print(f"Query time: {time() - start:.2f} seconds")

  1%|          | 1/100 [00:00<00:48,  2.04it/s]

Query time: 0.49 seconds


  2%|▏         | 2/100 [00:00<00:43,  2.28it/s]

Query time: 0.40 seconds


  3%|▎         | 3/100 [00:01<00:34,  2.84it/s]

...

Query time: 0.20 seconds


 99%|█████████▉| 99/100 [00:25<00:00,  4.96it/s]

Query time: 0.22 seconds


100%|██████████| 100/100 [00:25<00:00,  3.95it/s]

Query time: 0.28 seconds


In [ ]:


results = {}
for fn in ['add_headers_and_previous_7_sentences']:
    enricher = get_enricher(fn)
    ranks = []
    for i in tqdm(range(len(examples))):
        example = examples.iloc[i]
        ranks += get_ranks(
            example=example, 
            embedder=embedder, 
            enricher=enricher,
            table_name='bge', 
            top_k=340000)
    results[fn] = ranks
    series = pd.Series(ranks)
    print(f"Enrichment function: {fn}.")
    print(f"Stats: {series.describe()}")
# Save results
df = pd.DataFrame(results)
df.to_csv('ranks3.csv', index=False)





100%|██████████| 100/100 [06:47<00:00,  4.08s/it]

Enrichment function: add_headers_and_previous_7_sentences.
Stats: count      112.000000
mean      3136.544643
std       7961.988777
min          0.000000
25%         38.250000
50%        373.000000
75%       2512.750000
max      52857.000000
dtype: float64


In [4]:
df1 = pd.read_csv('ranks.csv')
df2 = pd.read_csv('ranks2.csv')
df3 = pd.read_csv('ranks3.csv')
df_combined = pd.concat([df1, df2, df3], axis=1)
df_combined.head()

,identity,add_abstract,add_title,add_title_and_abstract,add_previous_3_sentences,add_previous_7_sentences,add_headers_and_previous_3_sentences,add_headers_and_previous_7_sentences
0,34,1037,6,545,259,2974,1074,568
1,128,44,599,539,216,230,847,865
2,7565,179,175,18,3,57,13,20
3,1869,235,305,303,629,2489,458,375
4,23,3700,119,2313,651,7159,3887,3080


In [ ]:
df = pd.read_csv('ranks.csv')
df.add_title.describe()
df.add_abstract.describe()

In [ ]:
# Try again at 100,000 top k
ranks = []
for i in tqdm(range(len(examples))):
    example = examples.iloc[i]
    ranks += get_ranks(example=example, table_name='bge', top_k=250000)

print(ranks)

len([rank for rank in ranks if rank == -1])

In [ ]:
print(f"Total number of ranks: {len(ranks)}")

In [ ]:
import matplotlib.pyplot as plt

#
plt.hist(ranks, bins=100)
plt.show()